In [43]:
# ---------------------------------------------------------
# 0. Imports
# ---------------------------------------------------------
import pandas as pd

from hierarchicalforecast.utils import aggregate
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.methods import MinTrace
from hierarchicalforecast.evaluation import evaluate
from utilsforecast.losses import rmse

# ---------------------------------------------------------
# 1. Carregar dados de treino e teste
#    (antes de 2024 = treino, 2024 = teste)
# ---------------------------------------------------------
train_path = "hts2_train_df.csv"  # ajuste para seu caminho real
test_path  = "hts2_test_df.csv"

train = pd.read_csv(train_path)
test  = pd.read_csv(test_path)

# Converte data
for df in (train, test):
    df["date"] = pd.to_datetime(df["date"])

# Se você tiver um único CSV e quiser separar por data, seria algo como:
# data = pd.read_csv("todos_os_dados.csv")
# data["date"] = pd.to_datetime(data["date"])
# train = data[data["date"] < "2024-01-01"].copy()
# test  = data[data["date"] >= "2024-01-01"].copy()

# ---------------------------------------------------------
# 2. Preparar dados no formato esperado pelo HierarchicalForecast
#    - Ignoramos linhas onde continent == 'TOTAL' e deixamos o pacote
#      gerar o TOTAL pela agregação.
#    - Criamos uma coluna de nível superior ("World") constante.
#    - Renomeamos colunas para o padrão: ds (tempo), y (target)
#      e BaseModel (previsão do modelo base).
# ---------------------------------------------------------
def prepare_bottom_level(df: pd.DataFrame) -> pd.DataFrame:
    # Mantém apenas continentes (bottom level)
    df = df[df["continent"] != "TOTAL (Agregado)"].copy()

    df["World"] = "WORLD"  # nível mais agregado

    df = df.rename(
        columns={
            "date": "ds",
            "continent": "Continent",
            "actual": "y",          # alvo
            "forecast": "BaseModel" # previsões base do seu modelo
        }
    )
    return df[["World", "Continent", "ds", "y", "BaseModel"]]

bottom_train = prepare_bottom_level(train)
bottom_test  = prepare_bottom_level(test)

# ---------------------------------------------------------
# 3. Construir a hierarquia (Total -> Continente) com aggregate
#    spec define os níveis:
#    - ["World"]                        -> série TOTAL
#    - ["World", "Continent"]          -> séries por continente
# ---------------------------------------------------------
hierarchy_levels = [
    ["World"],
    ["World", "Continent"]
]

# Treino: contém y (actual) + BaseModel (previsão 1-step-ahead/insample)
Y_train_df, S_df, tags = aggregate(
    df=bottom_train,
    spec=hierarchy_levels,
    target_cols=["y", "BaseModel"]  # agrega ambos
)

# Teste: contém y (valor observado no teste) + BaseModel (previsão out-of-sample)
Y_test_df, _, _ = aggregate(
    df=bottom_test,
    spec=hierarchy_levels,
    target_cols=["y", "BaseModel"]
)

# ---------------------------------------------------------
# 4. Separar dataframes no formato esperado pela reconciliação
#    - Y_df: dados "históricos" + previsões insample para estimar covariância
#            (aqui estamos usando treino, com colunas y e BaseModel)
#    - Y_hat_df: previsões base no período de teste (futuro)
#            (coluna BaseModel será reconciliada)
# ---------------------------------------------------------
Y_df = Y_train_df.copy()  # contém: unique_id, ds, y, BaseModel

# Y_hat_df: previsões para o horizonte futuro (teste).
# O HierarchicalForecast só precisa das colunas: unique_id, ds e modelos (BaseModel).
Y_hat_df = Y_test_df[["unique_id", "ds", "BaseModel"]].copy()

# ---------------------------------------------------------
# 5. Definir o reconciliador MinT (MinTrace com mint_shrink)
#    Isso é o equivalente ao MinT do FPP3.
# ---------------------------------------------------------
reconcilers = [
    MinTrace(method="mint_shrink"),  # MinT(Shrink),
    MinTrace(method="mint_cov"),
    MinTrace(method="ols")
]

hrec = HierarchicalReconciliation(reconcilers=reconcilers)

# ---------------------------------------------------------
# 6. Rodar a reconciliação
#    - Y_hat_df: previsões base (teste)
#    - Y_df: dados com y e BaseModel no treino (para estimar matriz de covariância)
#    - S_df, tags: estrutura da hierarquia
# ---------------------------------------------------------
Y_rec_df = hrec.reconcile(
    Y_hat_df=Y_hat_df,
    Y_df=Y_df,
    S=S_df,
    tags=tags
)

# O resultado terá colunas:
#   - "BaseModel" (previsão base incoerente)
#   - "BaseModel/MinTrace_method-mint_shrink" (previsão reconciliada MinT)

print(Y_rec_df.tail(10))

# ---------------------------------------------------------
# 7. (Opcional) Juntar com os valores reais de teste e avaliar RMSE
# ---------------------------------------------------------
# Juntando previsões reconciliadas com os valores reais do período de teste
# (Y_test_df tem a coluna "y" com os valores observados nesse período)
Y_eval = Y_rec_df.merge(
    Y_test_df[["unique_id", "ds", "y"]],
    on=["unique_id", "ds"],
    how="left"
)

# Definir tags de avaliação por nível (Total e Continentes)
eval_tags = {
    "Total":     tags["World"],               # nível agregado (WORLD)
    "Continents": tags["World/Continent"],    # nível por continente
}

evaluation = evaluate(
    df=Y_eval,
    metrics=[rmse],
    tags=eval_tags,
    train_df=Y_df,  # opcional, útil p/ métricas tipo MASE
    id_col="unique_id",
    time_col="ds",
    target_col="y"
)

#print(evaluation)


       unique_id         ds     BaseModel  \
38  WORLD/Europa 2024-03-01  32191.378591   
39  WORLD/Europa 2024-04-01  24225.701114   
40  WORLD/Europa 2024-05-01  16058.418073   
41  WORLD/Europa 2024-06-01  16279.466939   
42  WORLD/Europa 2024-07-01  23906.166377   
43  WORLD/Europa 2024-08-01  24266.246572   
44  WORLD/Europa 2024-09-01  18274.542567   
45  WORLD/Europa 2024-10-01  23508.461584   
46  WORLD/Europa 2024-11-01  24836.935039   
47  WORLD/Europa 2024-12-01  32739.487808   

    BaseModel/MinTrace_method-mint_shrink  BaseModel/MinTrace_method-mint_cov  \
38                           32191.378591                        32191.378591   
39                           24225.701114                        24225.701114   
40                           16058.418073                        16058.418073   
41                           16279.466939                        16279.466939   
42                           23906.166377                        23906.166377   
43                

In [44]:
Y_rec_df

,unique_id,ds,BaseModel,BaseModel/MinTrace_method-mint_shrink,BaseModel/MinTrace_method-mint_cov,BaseModel/MinTrace_method-ols
0,WORLD,2024-01-01,36891.923514,36891.923514,36891.923514,36891.923514
1,WORLD,2024-02-01,40523.401779,40523.401779,40523.401779,40523.401779
2,WORLD,2024-03-01,36846.981231,36846.981231,36846.981231,36846.981231
3,WORLD,2024-04-01,27729.357809,27729.357809,27729.357809,27729.357809
4,WORLD,2024-05-01,19588.255734,19588.255734,19588.255734,19588.255734
5,WORLD,2024-06-01,20273.692410,20273.692410,20273.692410,20273.692410
6,WORLD,2024-07-01,27848.930334,27848.930334,27848.930334,27848.930334
7,WORLD,2024-08-01,28662.223275,28662.223275,28662.223275,28662.223275
8,WORLD,2024-09-01,22075.544869,22075.544869,22075.544869,22075.544869
9,WORLD,2024-10-01,27064.284388,27064.284388,27064.284388,27064.284388
